In [ ]:
#|default_exp vision.learner

In [ ]:
#| export
from __future__ import annotations

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *
from fastgs.vision.core import *

# Multi-spectral Vision Learner

> Learner support functions for working with `TensorImageMS`


## Show functions -

In [ ]:
#| export
def _show_one_result(img: TensorImageMS, msk: TensorMask, out, row, mskovl, **kwargs):
    if mskovl:
        rowl = len(row) / 2
        return [msk.show(ctx=c, **kwargs) for c in img.show(row[:rowl])]
        + [out.show(ctx=c, **kwargs) for c in img.show(row[rowl:])]
    else:
        nimgs = img.num_images()
        return img.show(ctxs=row[:nimgs]) + [msk.show(row[nimgs])] + [out.show(row[nimgs+1])]

In [ ]:
#|export
@typedispatch
def show_results(x:TensorImageMS, y:TensorMask, samples, outs, ctxs=None, max_n=6,
                 nrows:int=None, ncols:int=None, figsize=None, mskovl:bool=True, **kwargs):
    assert nrows is None and ncols is None and ctxs is None
    
    nimgs = x.num_images()
    n = min(len(samples),max_n)
    nrows = 2 * n if mskovl else n
    ncols = nimgs if mskovl else nimgs + 2

    ctxs = get_grid(nrows * ncols, nrows, ncols, figsize=figsize, title='Ground Truth/Prediction')
    chksize = 2 * n if mskovl else nimgs + 2
    rwcx = [ctxs[pos : pos + chksize] for pos in range(0, len(ctxs), ncols)]

    imgs,msks,otps = samples.itemgot(0),samples.itemgot(1),outs.itemgot(0)
    return [_show_one_result(img, msk, otp, row, mskovl, **kwargs) for img,msk,otp,row in zip(imgs, msks, outs, rwcx)]

In [ ]:
#|export
@typedispatch
def plot_top_losses(x:TensorImageMS, y:TensorMask, samples, outs, raws, losses, nrows=None, ncols=None, figsize=None, **kwargs):
    axes = get_grid(len(samples)*3, nrows=len(samples), ncols=3, figsize=figsize, flatten=False, title="Input | Target | Prediction")
    if axes.ndim == 1: axes = (axes,)
    titles = ["input", "target", "pred"]
    for axs,s,o,l in zip(axes, samples, outs, losses):
        imgs = (s[0], s[1], o[0])
        for ax,im,title in zip(axs, imgs, titles):
            if title=="pred": title += f"; loss = {l:.4f}"
            im.show(ctx=ax, **kwargs)
            ax.set_title(title)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()